In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, date, time, timedelta
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
#importing over and undersampling algorithms from imblearn (you will have to manually install it in your envoirenment with pip install imblearn) 
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing

In [16]:
data = pd.read_csv('./data/training.csv')


# Convert time to datatime format
data['TransactionStartTime'] = pd.to_datetime(data['TransactionStartTime'], format='%Y-%m-%dT%H:%M:%SZ')
data['Hour'] = data['TransactionStartTime'].dt.hour
#data['Month'] = data['TransactionStartTime'].dt.month

#Creating a new variable
data.loc[data['Amount'] >= 0, 'DirectionOfMoney'] = 0
data.loc[data['Amount'] < 0, 'DirectionOfMoney'] = 1

#Creating the final dataset

le = preprocessing.LabelEncoder()


# choose categorical variables
cat_var = [ 'ProviderId', 'ProductId', 'ChannelId', 'ProductCategory', 'PricingStrategy', 'Hour'] # '

columns = data[cat_var].columns.tolist()

for each in columns:
    data[each] = le.fit_transform(data[each])#

df = data.drop(['FraudResult','Amount','TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'CountryCode', 'TransactionStartTime'], axis=1,)
df = pd.get_dummies(df, columns=cat_var)

df['ProductId_23'] = 0
df['ProductId_24'] = 0
df['ProductId_25'] = 0
df['ChannelId_4'] = 0

#defining X and y
X = df
y = data['FraudResult']
X.to_csv("data/X_.csv")  
y.to_csv("data/y_.csv") 
X.columns

Index(['Value', 'DirectionOfMoney', 'ProviderId_0', 'ProviderId_1',
       'ProviderId_2', 'ProviderId_3', 'ProviderId_4', 'ProviderId_5',
       'ProductId_0', 'ProductId_1', 'ProductId_2', 'ProductId_3',
       'ProductId_4', 'ProductId_5', 'ProductId_6', 'ProductId_7',
       'ProductId_8', 'ProductId_9', 'ProductId_10', 'ProductId_11',
       'ProductId_12', 'ProductId_13', 'ProductId_14', 'ProductId_15',
       'ProductId_16', 'ProductId_17', 'ProductId_18', 'ProductId_19',
       'ProductId_20', 'ProductId_21', 'ProductId_22', 'ChannelId_0',
       'ChannelId_1', 'ChannelId_2', 'ChannelId_3', 'ProductCategory_0',
       'ProductCategory_1', 'ProductCategory_2', 'ProductCategory_3',
       'ProductCategory_4', 'ProductCategory_5', 'ProductCategory_6',
       'ProductCategory_7', 'ProductCategory_8', 'PricingStrategy_0',
       'PricingStrategy_1', 'PricingStrategy_2', 'PricingStrategy_3', 'Hour_0',
       'Hour_1', 'Hour_2', 'Hour_3', 'Hour_4', 'Hour_5', 'Hour_6', 'Hour_7',
      

In [27]:
condition1 = df['ProductCategory_2'] == 1
condition2 = df['DirectionOfMoney'] == 0.0
condition3 = df['ChannelId_2'] == 1
condition4 = (df['PricingStrategy_0'] == 1) | (df['PricingStrategy_2'] == 1)
condition5 = df["ProductId_6"] == 1
condition6 = (df['ProviderId_1'] == 1) | (df['ProviderId_3'] == 1)| (df['ProviderId_5'] == 1)
condition7 = df['Value'] >= 200000
predictions = condition1 & condition2 & condition3 & condition4 & condition5 & condition6 & condition7
pred = []
for i in predictions:
    if i == True:
        pred.append(1)
    else:
        pred.append(0)
#printing scores for baseline
print_evaluations(y, predictions)
print(sum(pred))

Accuracy: 0.9979824799816019
Precision: 0.0
Recall: 0.0
F1: 0.0
0


/Users/corneliusbohle/Desktop/neuefische/fraud/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
def print_evaluations(y,predictions):
    print(f"Accuracy: {accuracy_score(y, predictions)}")
    print(f"Precision: {precision_score(y, predictions)}")
    print(f"Recall: {recall_score(y, predictions)}")
    print(f"F1: {f1_score(y, predictions)}")